<a href="https://colab.research.google.com/github/CS23M005/Assignment2_PartA/blob/main/CS23M005_A2_PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
device

device(type='cuda')

In [3]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-06 15:55:32--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.207, 142.250.1.207, 108.177.120.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: 'nature_12K.zip'

nature_12K.zip      100%[===================>]   3.55G   268MB/s    in 13s     

2024-04-06 15:55:45 (278 MB/s) - 'nature_12K.zip' saved [3816687935/3816687935]



In [4]:
!rm nature_12K.zip

In [6]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models


In [7]:
def getOptim(model,optim_name, learning_rate):
  if(optim_name == 'sgd'):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
  elif(optim_name == 'adam'):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  else:
    optimizer = optim.NAdam(model.parameters(), lr=learning_rate)
  return optimizer


In [8]:
def check_accuracy(loader,model,criterion,batchSize):
    num_correct = 0
    num_loss = 0
    total = 0
    num_samples = 0
    total_loss = 0.0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x)
            loss = criterion(scores, y)
            total_loss += loss.item()*batchSize
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)
    model.train()
    return (num_correct / num_samples)*100 , total_loss

In [9]:
    transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])

    train_dataset = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform)

    train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[8000,1999])
    
    transform2 = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])

    train_dataset2 = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform2)

    train_dataset2,val_dataset2 = torch.utils.data.random_split(train_dataset2,[8000,1999])
    
    def getData(data_aug, batchSize):
        if(data_aug == "no"):
            train_loader = torch.utils.data.DataLoader(train_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
            val_loader = torch.utils.data.DataLoader(val_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
        else:
            train_loader = torch.utils.data.DataLoader(train_dataset2,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
            val_loader = torch.utils.data.DataLoader(val_dataset2,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
        return train_loader, val_loader

In [17]:
def resnet50_ud(output_size):
    model = models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = torch.nn.Linear(num_features, output_size)
    for p in model.parameters():
        p.requires_grad = False
    for p in model.fc.parameters():
        p.requires_grad = True
    return model

In [21]:
def train_cnn_ud(output_size,optim_name,batchSize,num_epochs,learning_rate, data_aug):

    train_loader, val_loader = getData(data_aug, batchSize)
    model = resnet50_ud(output_size).to(device)
    optimizer = getOptim(model,optim_name, learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            data = data.to(device=device)
            targets = targets.to(device=device)
            scores = model(data)
            loss = criterion(scores,targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_accuracy,train_loss = check_accuracy(train_loader, model,criterion,batchSize)
        validation_accuracy,validation_loss = check_accuracy(val_loader, model,criterion,batchSize)
        print(f"train_accuracy:{train_accuracy:.4f},train_loss:{train_loss:.4f}")
        print(f"validation_accuracy:{validation_accuracy:.4f},validation_loss:{validation_loss:.4f}")
        wandb.log({'train_accuracy':train_accuracy})
        wandb.log({'train_loss':train_loss})
        wandb.log({'val_accuracy':validation_accuracy})
        wandb.log({'val_loss':validation_loss})

    #wandb.log({'train_accuracy':train_accuracy})


In [22]:
optim_name = 'adam'
batchSize=32
dropOut = 0.1
num_epochs = 5
learning_rate = 1e-3
input_channel=3
output_size=10
num_filters=16
filter_size=3
activation_fun = "relu"
filter_config = "same"
stride = 1
poolstride = 2
poolsize = 2
data_aug = "no"
train_cnn_ud(output_size,optim_name,batchSize,num_epochs,learning_rate, data_aug)

100%|██████████| 250/250 [00:46<00:00,  5.35it/s]


train_accuracy:75.5125,train_loss:5980.1779
validation_accuracy:73.0365,validation_loss:1675.5604


NameError: name 'wandb' is not defined

In [24]:
!pip install wandb
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def main_fun():
    wandb.init(project ='Assignment2_PartB')
    params = wandb.config
    with wandb.init(project = 'Assignment2_PartB', name='optim_'+str(params.optim_name)
                    +'epochs'+str(params.num_epochs) + 'batch_size_'+str(params.batchSize)
                    +'lear_rate_'+str(params.learning_rate) + 'data_aug_'+ str(params.data_aug)) as run:
        train_cnn_ud(output_size,params.optim_name,params.batchSize,params.num_epochs,params.learning_rate, params.data_aug)

sweep_params = {
    'method' : 'bayes',
    'name'   : 'cs23m005',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
        'optim_name' :{'values':['sgd','adam','nadam']},
        'batchSize' : {'values':[32,64]},
        'data_aug' :{'values':['yes','no']},
        'num_epochs':{'values':[5,10]},
        'learning_rate' :{'values':[1e-3,1e-4]}
    }
}
sweepId = wandb.sweep(sweep_params,project = 'Assignment2_PartB')
wandb.agent(sweepId,function =main_fun,count = 10)
wandb.finish()

Create sweep with ID: 7687mpbh
Sweep URL: https://wandb.ai/cs23m005/Assignment2_PartB/sweeps/7687mpbh


wandb: Agent Starting Run: xei231sm with config:
wandb: 	batchSize: 32
wandb: 	data_aug: no
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 10
wandb: 	optim_name: nadam
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
